In [1]:
# MAIN PACKAGES
import pandas as pd
import numpy as np
import sklearn
from tqdm import tqdm

# MODEL VALIDATION
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

#RFR SPECIFIC
from sklearn.ensemble import RandomForestRegressor


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


This notebook reads the top encoding datasets and uses them to form predictions on the Novel mutant library and saves predictions of the test set for cross validation.


Inputs:
1.   Combined Dataset Path (String)
2.   Desired Name of Dependent Variable (String)
3.   len(Sequence of Base Variant) (int)
4.   Property Matrix Path (String)
5.   path to parent folder
5.   path to RFR/RFR_Encoding_Datasets.csv
6.   path to Novel mutant library

Outputs:
5 novel prediction notebooks
*   once per top five encoding datasets, the final predictions of the novel library are saved.
*   Saved as dataset_name_novel_library_predictions.csv in the RFR/ Sub Folder

5 test set prediction notebooks
*   once per top five encoding datasets, the final predictions of test set are saved.
*   Saved as dataset_name_test_set_predictions.csv in the RFR/ Sub Folder


In [9]:
mutation_datset_path = '/content/drive/MyDrive/ml_paper_ipnys/backend_data/combined_dataset.csv'
dependent_variable = 'Fluor Decay' # Desired Name of Dependent Variable
len_sequence_of_base_variant = 451 # len(Sequence of Base Variant)
encoding_dataset_path = '/content/drive/MyDrive/ml_paper_ipnys/backend_data/full_property_matrix.csv'
parent_folder_path = '2023-07-22-ensemble-run/' #should be created in wrapper

# specific for this notebook
novel_library_path = '/content/drive/MyDrive/ml_paper_ipnys/backend_data/novel_mutant_library.csv'
top_encoding_dataset_path = '/content/drive/MyDrive/ml_paper_ipnys/RFR_Encoding_Datasets.csv'

In [4]:
data = pd.read_csv(mutation_datset_path)
position_cols = np.arange(0,len_sequence_of_base_variant)
position_cols = [str(i) for i in position_cols]
encoded_df = data[position_cols]

x_train, x_test, y_train, y_test = train_test_split(encoded_df,
                                                    data[dependent_variable],
                                                    test_size=0.20,
                                                    random_state=42)

In [5]:
# Read in encoding dataset
encoding_data = pd.read_csv(encoding_dataset_path, index_col = 0)

In [13]:
#read in the Library of Novel Variant Sequences
data_pred = pd.read_csv(novel_library_path, index_col = 0)

#Remove Any Duplicate Sequences (redundancies with 7s)
data_pred = data_pred.drop_duplicates(subset = data_pred.columns[2:-2],
                                      keep='first',
                                      inplace=False, ignore_index=False)
dropped = data_pred[data_pred.duplicated(subset = data_pred.columns[2:-2])]

#tests
assert len(dropped) == 0

In [10]:
# read in the encoding datasets:
top_encoding_data = pd.read_csv(top_encoding_dataset_path,index_col= 0)
top_encoding_data = top_encoding_data.sort_values(by = 'Test Set R Squared',
                                                          ascending = False)
names = [e[-10:] for e in top_encoding_data['Encoding Dataset'][0:5]]


##tests
assert np.shape(names) == (5,)
assert top_encoding_data['Test Set R Squared'].iloc[0]>=top_encoding_data['Test Set R Squared'].iloc[1]

In [10]:
#initialize output dataframe
df_RFR = pd.DataFrame()
save_path = parent_folder_path +'RFR/'

#Iterate through every encoding dataset + train model +
#record performance
for AA_property_dataset in tqdm(names,
                                desc = 'Property Datasets Encoded:'):

    #format to match column names
    column_names_ref = 'AAindex: '+AA_property_dataset

    # Make the train/test be on a copy to ensure there
    #is no data overwriting within loop
    x_train_copy = x_train.copy()
    x_test_copy = x_test.copy()
    x_pred_copy = data_pred.copy()

    #extract encoding data for specific iteration
    volume_dict = {'Amino Acid Code': encoding_data[encoding_data.columns[0]],
                   AA_property_dataset: encoding_data[column_names_ref]}
    volume_data = pd.DataFrame(volume_dict)


    #Some encoding datasets contain NaNs, I skip these
    #datasets since theyre incomplete
    #Next three lines check the encoding data for NaNs
    df = list(volume_data.iloc[:,1].values)
    T = np.isnan(df)
    TF = True in T

    #If there is no NaN, perform model training
    if TF == False:

        #initialize list to append to throughout training
        interlist = []

        #Use volume_data as a codex to translate sequence data...
        # amino acids will translate to float type data
        col_title = volume_data.columns[1]
        for row, sample in enumerate(volume_data['Amino Acid Code']):
            amino = sample
            replacement_value = float(volume_data[col_title].iloc[row])
            x_train_copy = x_train_copy.replace(amino,replacement_value)
            x_test_copy = x_test_copy.replace(amino,replacement_value)
            x_pred_copy = x_pred_copy.replace(amino,replacement_value)


        #Initialize Model and fit data to extract feature importances
        model = RandomForestRegressor()
        model.fit(x_train_copy,y_train)
        feat_importances = pd.Series(model.feature_importances_,
                                     index=x_train_copy.columns)


        ##Hyper Parameter Tuning
        #Grid Search approach, I test every iteration to find the best
        #combination of neighbors and features

        #Initialize the output lists to append to
        n_est = []
        test_r2s = []

        #For 21 possible best features
        for l in range(21):
            if l > 0:
                cols = list(feat_importances.nlargest(l).index)

                #extract l features from X_train/X_test
                x_train_copy_ = x_train_copy[cols]
                x_test_copy_ = x_test_copy[cols]

                #Initialize list of estimators to test
                n_estimators = [10, 15, 20, 25, 30, 35, 40,
                                45, 50, 55, 65, 75, 85, 100]

                #Initialize the output lists to append to
                n_mse_list = []
                n_r2_list = []

                for estimators in n_estimators:
                    #initialize model with n number of estimators
                    clf_RF = RandomForestRegressor(n_estimators=estimators,
                                                   random_state=42)
                    #Fit the Train data
                    clf_RF.fit(x_train_copy_, y_train)
                    #Predict the Test set and generate metrics of fit
                    y_RF = clf_RF.predict(x_test_copy_)
                    n_mse_list.append(mean_squared_error(y_test, y_RF))
                    n_r2_list.append(sklearn.metrics.r2_score(y_test, y_RF))

                #find which number of estimators led to the best R2 for
                # the test set
                best_est = n_estimators[n_mse_list.index(min(n_mse_list))]
                n_est.append(best_est)
                test_r2s.append(np.mean(n_r2_list))

        #find which iteration led to the greatest R2
        #est_best is the number of estimators when the
        #max R2 occured
        best_feats = test_r2s.index(max(test_r2s))
        est_best = n_est[best_feats]

        #similarly, extract the optimal number of features
        #by extracting the number of features that led to the
        #greatest R2
        cols = list(feat_importances.nlargest(best_feats + 1).index)
        x_train_copy_ = x_train_copy[cols]
        x_test_copy_ = x_test_copy[cols]
        x_pred_copy_ = x_pred_copy[cols]

        #initialize new model + fit data
        clf = RandomForestRegressor(n_estimators = est_best,random_state = 42)
        clf.fit(x_train_copy_, y_train)

        #Create cross validation prediction
        y_pred = clf.predict(x_test_copy_)

        #Save the overall R2 for the tuned model
        r2 = sklearn.metrics.r2_score(y_test, y_pred)

        #append the data to a dataframe for export
        inter_df= pd.DataFrame({'Encoding Dataset': [AA_property_dataset[-11:]],
                                'Test Set R Squared' : [r2]})

        df_RFR = pd.concat([df_RFR,inter_df],ignore_index = True)


        # Novel Library Predictions
        y_pred_new = clf.predict(x_pred_copy_)
        data_pred[dependent_variable +' Predicted'] = y_pred_new
        data_pred = data_pred.sort_values(by = dependent_variable +' Predicted',
                                          ascending = False)
        newmut = pd.DataFrame(data_pred)


        #Test Set Predictions
        test_set_df = pd.DataFrame()
        test_set_df['True'] = y_test
        test_set_df['Predicted'] = y_pred

        #file saving
        newmut.to_csv(save_path + AA_property_dataset+'_novel_library_predictions.csv')
        test_set_df.to_csv(save_path + AA_property_dataset+'_test_set_predictions.csv')


        #FOR COLAB ONLY
        save_path = '/content/drive/MyDrive/ml_paper_ipnys/RFR_'
        newmut.to_csv(save_path + AA_property_dataset+'_novel_library_predictions.csv')
        test_set_df.to_csv(save_path + AA_property_dataset+'_test_set_predictions.csv')

assert len(df_RFR) == 5
assert len(data_pred) == len(newmut)
assert len(test_set_df) == len(x_test)

Property Datasets Encoded:: 100%|██████████| 5/5 [02:45<00:00, 33.12s/it]


In [11]:
df_RFR.sort_values(by='Test Set R Squared', ascending = False)

,Encoding Dataset,Test Set R Squared
0,BASU050103,0.847938
4,ZIMJ680103,0.841128
3,GEIM800104,0.839672
1,HUTJ700101,0.822306
2,ISOY800107,0.778070
